In [ ]:
import itertools as it
import os

os.environ["PRIVBAYES_BIN"] = "./ydnpd/synthesis/privbayes"

import ray
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import display, Markdown

import ydnpd

## Tasks

In [ ]:
total_task_size = 0
for idx, task in enumerate(ydnpd.span_utility_tasks(), start=1):
    print(f"{idx:2} {task}")
    total_task_size += task.size()
print(f"\nTotal task size: {total_task_size}")

## Execute Tasks

In [ ]:
ray.init(num_cpus=3)

In [ ]:
utility_tasks = ydnpd.span_utility_ray_tasks(with_wandb=False)
utility_tasks_results = ray.get(utility_tasks)
utility_tasks_results = list(it.chain(*utility_tasks_results))

## Utility-Related Tasks

In [ ]:
for experiments_name, experiments in ydnpd.config.ALL_EXPERIMENTS.items():

    display(Markdown(f"## {experiments_name.upper()}"))

    for metric in ydnpd.EVALUATION_METRICS:
        if metric not in utility_tasks_results[0]["evaluation"]:
            continue

        display(Markdown(f"### {metric}"))

        gs = ydnpd.UtilityTask.plot(utility_tasks_results, experiments, metric=metric)

        for g in gs:
            display(g.fig)
            plt.close(g.fig)

    for metric in ydnpd.EVALUATION_METRICS:
        if metric not in utility_tasks_results[0]["evaluation"]:
            continue

        display(Markdown(f"### {metric}"))
        with pd.option_context("display.max_rows", None):
            display(
                ydnpd.UtilityTask.evaluate(utility_tasks_results, experiments, metric)
                .multiply(100)
                .round(1)
            )